In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow.compat.v2.feature_column as fc
import matplotlib.pyplot as plt
import sys
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [8]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# Single Layer

In [9]:
x = tf.placeholder("float", [None, 784])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [10]:
y = tf.nn.softmax(tf.matmul(x,W)+b)
y_ = tf.placeholder("float", [None,10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y)) #loss function
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy) #choose optimizer and load loss function

In [11]:
#init all var
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [12]:
# train data
for i in range(500): #epoch
    batch_xs, batch_ys = mnist.train.next_batch(100) #generate 100 data for one epoch
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys}) #feed 

In [13]:
# test
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9085


# Multi layer

In [15]:
def conv2d(x, output_channel, kernel, strides):
    """
    input kernel size [length, width]
    strides int
    """
    #kernal size for 2d [length, width, channel, features]
    kernel_shape = [kernel[0], kernel[1], x.get_shape()[-1].value, output_channel]
    initial_W = tf.truncated_normal(kernel_shape, stddev=0.1) #std some what IMPORTANT
    W = tf.Variable(initial_W)
    
    #Bias size = output_channel
    initial_B = tf.constant(0.1, shape=[output_channel])
    B = tf.Variable(initial_B)
    #build layer + Bias
    layer = tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")

    #use activation func
    conv_act_layer = tf.nn.relu(layer + B)

    return conv_act_layer

In [16]:
def max_pool(x, kernel, strides):
    """
    input kernel size [length, width]
    strides int
    """
    max_pool_layer = tf.nn.max_pool(x, 
                       ksize=[1, kernel[0], kernel[1], 1], 
                       strides= [1, strides, strides, 1], 
                       padding="SAME")

    return max_pool_layer

In [17]:
def dense_layer(x, output_channel):
    # weight
    shape = [x.get_shape()[-1].value, output_channel]
    initial_W = tf.truncated_normal(shape, stddev=0.1) #std some what IMPORTANT
    W = tf.Variable(initial_W)
    #bias
    initial_B = tf.constant(0.1, shape=[output_channel])
    B = tf.Variable(initial_B)

    layer = tf.matmul(x, W) # (W * x)
    act_layer = tf.nn.relu(layer + B)

    return act_layer

In [18]:
def softmax_layer(x, output_channel):
    # weight
    shape = [x.get_shape()[-1].value, output_channel]
    initial_W = tf.truncated_normal(shape, stddev=0.1) #std some what IMPORTANT
    W = tf.Variable(initial_W)
    #bias
    initial_B = tf.constant(0.1, shape=[output_channel])
    B = tf.Variable(initial_B)

    layer = tf.matmul(x, W)
    act_layer = tf.nn.softmax(layer+B)

    return act_layer

In [19]:
#input placeholder
x = tf.placeholder(dtype="float", shape=[None, 784])
input_x = tf.reshape(x, shape=[-1, 28, 28, 1])

#model network
conv2d_1 = conv2d(input_x, 32, [5,5], 1)
max_pool1 = max_pool(conv2d_1, [2,2], 2)
conv2d_2 = conv2d(max_pool1, 64, [5,5], 1)
max_pool2 = max_pool(conv2d_2, [2,2], 2)
flatten = tf.reshape(max_pool2, [-1, max_pool2.get_shape()[1]*max_pool2.get_shape()[2]*max_pool2.get_shape()[3]])
dense_1 = dense_layer(flatten, 1024)
output = softmax_layer(dense_1, 10) #output = 10

**Equation of cross entropy**  
L = label  
p = predicted probability  
n = classes  
$CE = \sum_{i=1}^n -(L* \log(p)) $

hint: $(L*\log(p))$ only the right class's probability will remain ex: $1*0.5 + 0*0.25 + 0*0.25$

In [20]:
#loss function
#softmax function predict the output probability 0 to 1, can use cross entropy to evaluate the performance
y_ = tf.placeholder("float", shape=[None, 10]) #label
cross_entropy = -1 * tf.reduce_sum(y_*tf.log(output)) #reduce_sum sum all the element

#add optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#test evaluation function (accuracy)
#tf.argmax will out put the "INDEX" of maxium value (the heightest probability)
#tf.equal will generate [True, True, False, ....] matrix

acc_matrix = tf.equal(tf.argmax(output, 1), tf.argmax(y_, 1))

#tf.cast will transform input to required type in here Bool to (0,1)
#tf.reduce_mean will output the mean of matrix

acc_eval = tf.reduce_mean(tf.cast(acc_matrix, "float"))

In [21]:
# Train
epoch = 500
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epoch):
        batch = mnist.train.next_batch(50)
        #visualize ACC every 50 epoch
        if i % 50 == 0:
            train_acc = acc_eval.eval(feed_dict={x: batch[0], y_:batch[1]})
            print(f"Steps{i}: ACC: {train_acc}")
        #train network
        train_step.run(feed_dict={x: batch[0], y_:batch[1]})
    test_acc = acc_eval.eval(feed_dict={x: mnist.test.images, y_:mnist.test.labels})
    print(f"Network ACC: {test_acc}")

Steps0: ACC: 0.03999999910593033
Steps50: ACC: 0.7799999713897705
Steps100: ACC: 0.9200000166893005
Steps150: ACC: 0.9200000166893005
Steps200: ACC: 0.9399999976158142
Steps250: ACC: 0.8399999737739563
Steps300: ACC: 0.8999999761581421
Steps350: ACC: 0.8999999761581421
Steps400: ACC: 0.9599999785423279
Steps450: ACC: 1.0
Network ACC: 0.9532999992370605
